Load Library

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.model_selection import cross_val_score

from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping

Load datasets

In [ ]:
data = datasets.load_digits()
X = data.data
y = data.target

### Neural Network Baseline

In [ ]:
def create_model(optimizer='sgd', neurons=32, batch_size=32, epochs=20, activation='relu', patience=3, loss='categorical_crossentropy'):
  model = Sequential()
  early_stopping = EarlyStopping(monitor='loss', patience=patience)

  model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
  model.add(Dense(neurons, activation=activation))

  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer=optimizer, loss=loss)

  history = model.fit(X, pd.get_dummies(y).values, batch_size=batch_size, epochs=epochs, callbacks=[early_stopping], verbose=0)
  return model

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0)
scores = cross_val_score(model, X, y, cv=3,scoring='accuracy')
print(f'Accuracy: {scores.mean()}')

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Accuracy: 0.9877573734001114


### Neural Network Optimization (Particle Swarm Optimization)

In [ ]:
!pip install optunity

     |████████████████████████████████| 4.6MB 4.2MB/s 
  Created wheel for optunity: filename=Optunity-1.1.1-cp37-none-any.whl size=72033 sha256=5d180692b282d93e333221c802cc14885eb48e85b4536ab5e8cfa728e524b10e
  Stored in directory: /root/.cache/pip/wheels/c9/e2/80/d3794524ae0042e147e035e132ec5fac09b8794b4acd94f046
Successfully built optunity


In [ ]:
import optunity
import optunity.metrics

In [ ]:
data = X
labels = y.tolist()

Search hyperparameter

In [ ]:
search = {
    'optimzer': [0, 3],
    'activation': [0, 1],
    'batch_size': [0, 2],
    'epochs': [20, 50],
    'neurons': [32, 64],
    'patience': [5, 10],
}

In [ ]:
@optunity.cross_validated(x=data, y=labels, num_folds=3)
def performance(x_train, y_train, x_test, y_test, optimzer=None, activation=None, batch_size=None, neurons=None, epochs=None, patience=None):
  if optimzer == 0:
    op = 'adam'
  elif optimzer == 1:
    op = 'sgd'
  else:
    op = 'rmsprop'

  if activation == 0:
    act = 'relu'
  else:
    act = 'tanh'

  if batch_size < 1:
    ba = 16
  else:
    ba = 32

  model = create_model(optimizer=op, activation=act, batch_size=ba, neurons=int(neurons), epochs=int(epochs), patience=int(patience))
  clf = KerasClassifier(build_fn=ANN, verbose=0)
  scores = np.mean(cross_val_score(clf, X, y, cv=3, scoring='accuracy'))

  return scores

In [ ]:
optimal_configuration, info, _ = optunity.maximize(performance, solver_name='particle swarm', num_evals=20, **search)
print(optimal_configuration)
print(f'MSE: {str(info.optimum)}')

/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


{'optimzer': 1.8779296875, 'activation': 0.8779296875, 'batch_size': 29.265625, 'epochs': 38.427734375, 'neurons': 51.71875, 'patience': 7.2021484375}
MSE: 0.989983305509182
